## Joining on Non-Equi Operators

In [15]:
import pandas as pd
import janitor
import numpy as np

In [16]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

df1 = pd.concat([df1]*50_000)
df2 = pd.concat([df2]*500)

In [17]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
...,...,...
1,1,5
2,1,7
3,2,1
4,2,3


In [18]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
...,...,...,...
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6


In [19]:
%%timeit
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>='),
        use_numba=True
    )

8.2 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>='),
        use_numba=False
    )

11 s ± 89.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Join on equi and non-equi operators is possible:

In [21]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        sort_by_appearance = True
    )

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:29: DeprecationWarning: The keyword argument 'sort_by_appearance' of 'conditional_join' is deprecated.
  return method(self._obj, *args, **kwargs)


left         right                  
            id value_1    id value_2A value_2B
0            1       5     1        3        5
1            1       5     1        3        5
2            1       5     1        3        5
3            1       5     1        3        5
4            1       5     1        3        5
...        ...     ...   ...      ...      ...
124999995    2       3     2        3        6
124999996    2       3     2        2        4
124999997    2       3     2        3        6
124999998    2       3     2        2        4
124999999    2       3     2        3        6

[125000000 rows x 5 columns]

The default join is inner. left and right joins are supported as well:

In [22]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='left',
        sort_by_appearance = True
    )

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:29: DeprecationWarning: The keyword argument 'sort_by_appearance' of 'conditional_join' is deprecated.
  return method(self._obj, *args, **kwargs)


left         right                  
            id value_1    id value_2A value_2B
0            1       2   NaN      NaN      NaN
1            1       5   1.0      3.0      5.0
2            1       5   1.0      3.0      5.0
3            1       5   1.0      3.0      5.0
4            1       5   1.0      3.0      5.0
...        ...     ...   ...      ...      ...
125099995    2       3   2.0      2.0      4.0
125099996    2       3   2.0      3.0      6.0
125099997    2       3   2.0      2.0      4.0
125099998    2       3   2.0      3.0      6.0
125099999    3       4   NaN      NaN      NaN

[125100000 rows x 5 columns]

In [23]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:29: DeprecationWarning: The keyword argument 'sort_by_appearance' of 'conditional_join' is deprecated.
  return method(self._obj, *args, **kwargs)


left         right                  
            id value_1    id value_2A value_2B
0          NaN     NaN     1        0        1
1          1.0     5.0     1        3        5
2          1.0     5.0     1        3        5
3          1.0     5.0     1        3        5
4          1.0     5.0     1        3        5
...        ...     ...   ...      ...      ...
125001495  2.0     3.0     2        3        6
125001496  2.0     3.0     2        3        6
125001497  2.0     3.0     2        3        6
125001498  2.0     3.0     2        3        6
125001499  NaN     NaN     3        1        3

[125001500 rows x 5 columns]

Join on just the non-equi joins is also possible:

In [24]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:29: DeprecationWarning: The keyword argument 'sort_by_appearance' of 'conditional_join' is deprecated.
  return method(self._obj, *args, **kwargs)


left         right                  
            id value_1    id value_2A value_2B
0            1       2     3        1        3
1            1       2     3        1        3
2            1       2     3        1        3
3            1       2     3        1        3
4            1       2     3        1        3
...        ...     ...   ...      ...      ...
124999995    3       4     2        3        6
124999996    3       4     1        3        5
124999997    3       4     2        3        6
124999998    3       4     1        3        5
124999999    3       4     2        3        6

[125000000 rows x 5 columns]

Join on not equal -> !=

In [25]:
df1.conditional_join(
        df2,
        ('id', 'id', "!=")
    )

left         right                  
            id value_1    id value_2A value_2B
0            1       2     2        0        1
1            1       2     2        2        4
2            1       2     2        3        6
3            1       2     2        0        1
4            1       2     2        2        4
...        ...     ...   ...      ...      ...
724999995    3       4     2        2        4
724999996    3       4     2        3        6
724999997    3       4     2        0        1
724999998    3       4     2        2        4
724999999    3       4     2        3        6

[725000000 rows x 5 columns]

If the columns from both dataframes have nothing in common, a single indexed column is returned:

In [26]:
(df1.select_columns('value_1')
    .conditional_join(
        df2.select_columns('val*'),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    )
)

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:29: FutureWarning: This function will be deprecated in a 1.x release. Please use `jn.select` instead.
  return method(self._obj, *args, **kwargs)


,value_1,value_2A,value_2B
0,2,1,3
1,2,1,3
2,2,1,3
3,2,1,3
4,2,1,3
...,...,...,...
124999995,4,3,6
124999996,4,3,5
124999997,4,3,6
124999998,4,3,5


Selection of relevant columns within `conditional_join`: 

In [27]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = 'id',
        right_columns = 'id'
    )

,left,right
,id,id
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2
...,...,...
349999995,2,3
349999996,2,3
349999997,2,3


Column renaming is also possible:

In [28]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = {'id':'df_id'},
        right_columns = {'id':'right_id'}
    )

,df_id,right_id
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2
...,...,...
349999995,2,3
349999996,2,3
349999997,2,3
349999998,2,3
